In [10]:
import pandas as pd

#upload CSV file
file_path = 'C:\capstoneproject\groceries_stock.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

  itemID        itemName  itemGroup   stock   price
0   P001  tropical fruit      fruit  1314.0  4298.0
1   P002      whole milk      drink   556.0  6214.0
2   P003       pip fruit      fruit  2183.0  9893.0
3   P004          carrot  vegetable  2012.0  7663.0
4   P005      rolls/buns       food   650.0  2052.0


<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\c'
C:\Users\jorda\AppData\Local\Temp\ipykernel_17512\1990069424.py:4: SyntaxWarning: invalid escape sequence '\c'
  file_path = 'C:\capstoneproject\groceries_stock.csv'


In [11]:
# To check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   itemID     38559 non-null  object 
 1   itemName   87 non-null     object 
 2   itemGroup  87 non-null     object 
 3   stock      38573 non-null  float64
 4   price      38573 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.5+ MB


In [12]:
# Renaming the columns, because it would be easier to understand what that means.
df.columns = ['itemID', 'itemName', 'itemGroup','stock', 'price']
df.head()

,itemID,itemName,itemGroup,stock,price
0,P001,tropical fruit,fruit,1314.0,4298.0
1,P002,whole milk,drink,556.0,6214.0
2,P003,pip fruit,fruit,2183.0,9893.0
3,P004,carrot,vegetable,2012.0,7663.0
4,P005,rolls/buns,food,650.0,2052.0


In [13]:
# Checking for the missing values
nan_values = df.isna().sum()
nan_values

itemID         206
itemName     38678
itemGroup    38678
stock          192
price          192
dtype: int64

In [14]:
# there are missing values
df = df.dropna(subset=['itemID','itemName', 'stock', 'price'])

df['stock'] = df['stock'].astype('int')
df['price'] = df['price'].astype('int')
df.info()  # They are in correct datatype now

<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 0 to 86
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   itemID     87 non-null     object
 1   itemName   87 non-null     object
 2   itemGroup  87 non-null     object
 3   stock      87 non-null     int64 
 4   price      87 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 4.1+ KB


In [15]:
# Counting the itemName based on itemGroup
item_counts = df.groupby('itemGroup')['itemName'].count()

# Displaying the counts
print(item_counts)

itemGroup
drink        17
food         35
fruit         6
meat         11
others       13
vegetable     5
Name: itemName, dtype: int64


In [16]:
def display_products(df):
    print("\nList of Saleable Products:")
    print(df)

def select_item_counts(df):
    item_group = input("Enter the item group: ").lower() 
    df['itemGroup'] = df['itemGroup'].str.lower()
    sku_counts = df[df['itemGroup'] == item_group].groupby('itemGroup').size().reset_index(name='count')
    
    if not sku_counts.empty:
        item_names = df[df['itemGroup'] == item_group]['itemName'].unique()
        print(f"Item names for item group '{item_group}': {', '.join(item_names)}")
    else:
        print(f"No SKUs found for item group '{item_group}'")
        return pd.DataFrame()  # Return an empty DataFrame if no SKUs are found
    
    return df[df['itemGroup'] == item_group]

def process_item_selection(df, cart):
    selected_group = select_item_counts(df)

    if not selected_group.empty:
        # Sorting the filtered DataFrame by itemGroup in descending order
        selected_group_sorted = selected_group.sort_values(by='itemGroup', ascending=False)
        print(selected_group_sorted)
        
        # Allow the user to select an item name
        item_name = input("Enter the item name: ").lower()
        selected_item = selected_group_sorted[selected_group_sorted['itemName'].str.lower() == item_name]
        
        if not selected_item.empty:
            stock = selected_item['stock'].sum()  # Assuming 'stock' is the column name for stock quantities
            price = float(selected_item['price'].values[0])  # Assuming 'price' is the column name for item prices
            # Allow the user to input a quantity
            user_quantity = int(input("Enter the quantity: "))

            if user_quantity <= stock:
                print("Your item is successfully added to cart")
                cart.append({'itemName': item_name, 'quantity': user_quantity, 'totalPrice': user_quantity * price})
            else:
                print("We are out of stock. Cannot proceed.")
        else:
            print(f"No data found for item name '{item_name}'")
    else:
        print("No data available for the selected item group.")

def process_payment(total_amount):
    print("Select payment method:")
    print("1. Cashless")
    print("2. Cash")
    payment_method = input("Enter the number of your payment method: ")

    if payment_method == "1":
        confirmation = input("Are you sure to proceed the payment? (Yes/Cancelled): ")
        if confirmation.lower() == "yes":
            print("Transaction successful!")
            return True
        elif confirmation.lower() == "cancelled":
            print("Transaction cancelled.")
            return False
        else:
            print("Invalid input. Transaction cancelled.")
            return False
    
    elif payment_method == "2":
        try:
            cash = float(input("Enter the amount of cash: "))
            if cash < total_amount:
                print("Transaction failed. Insufficient cash.")
                return False
            elif cash == total_amount:
                print("Transaction successful!")
                return True
            else:
                change = cash - total_amount
                print(f"Transaction successful! Your change is {change:.2f}")
                return True
        except ValueError:
            print("Invalid amount entered. Please enter a valid number.")
            return False
        
    else:
        print("Invalid payment method selected.")
        return False

def update_stock(df, cart):
    for item in cart:
        item_name = item['itemName']
        quantity_purchased = item['quantity']
        df.loc[df['itemName'] == item_name, 'stock'] -= quantity_purchased
    print("Stock updated successfully.")

def main(df):
    cart = []
    while True:
        print("\nMenu:")
        print("1. View Products")
        print("2. Make an Order")
        print("3. Exit")
        choice = input("Enter your choice: ")
        
        if choice == '1':
            display_products(df)
        elif choice == '2':
            process_item_selection(df, cart)
            add_more = input("Do you want to add more products? (yes/no): ").lower()
            if add_more != 'yes':
                if cart:
                    print("\n----- Bill -----")
                    print(f"{'Item':<15}{'Quantity':<10}{'Total Price':<10}")
                    print("-" * 35)
                    for item in cart:
                        print(f"{item['itemName'].capitalize():<15}{item['quantity']:<10}{float(item['totalPrice']):<10.2f}")
                    total_amount = sum(float(item['totalPrice']) for item in cart)
                    print("-" * 35)
                    print(f"{'Total Amount':<25}{total_amount:.2f}")

                    if process_payment(total_amount):
                        update_stock(df, cart)
                        print("\nUpdated stock:")
                        print(df)
                        break  # Stop the program after updating the stock
        elif choice == '3':
            print("Thank you for shopping!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main(df)


Menu:
1. View Products
2. Make an Order
3. Exit

List of Saleable Products:
   itemID                  itemName  itemGroup  stock  price
0    P001            tropical fruit      fruit   1314   4298
1    P002                whole milk      drink    556   6214
2    P003                 pip fruit      fruit   2183   9893
3    P004                    carrot  vegetable   2012   7663
4    P005                rolls/buns       food    650   2052
..    ...                       ...        ...    ...    ...
82   P083               white bread       food   2120   5435
83   P084  female sanitary products     others    450   2240
84   P085           cling film/bags     others   1892   6782
85   P086                      soap     others   2515   5935
86   P087            frozen chicken       meat   1861   5157

[87 rows x 5 columns]

Menu:
1. View Products
2. Make an Order
3. Exit
Item names for item group 'drink': whole milk, fruit/vegetable juice, oatmilk, bottled water, canned beer, coffee, misc

In [ ]:
# Initialize an empty DataFrame to store membership data
membership_df = pd.DataFrame(columns=['Name', 'Phone Number'])

def ask_membership():
    response = input("Do you want to create a membership? (yes/no): ").strip().lower()
    if response == 'yes':
        name = input("Please enter your name: ").strip()
        phone_number = input("Please enter your phone number: ").strip()
        # Add the new member to the DataFrame
        global membership_df
        membership_df = membership_df.append({'Name': name, 'Phone Number': phone_number}, ignore_index=True)
        print("Thank you for becoming a member!")
    else:
        print("Thank you for shopping with us!")

# Example usage after a transaction
ask_membership()

# Display the membership DataFrame
print(membership_df)